In [2]:
# MSM 107 analysis
# adcp data
# plotting adcp data

import pandas as pd
import numpy as np
import os, glob, re, itertools, math
import netCDF4
from matplotlib import pyplot as plt
from matplotlib import gridspec
from datetime import datetime

In [3]:
# path
adcppath = "/Users/dkim/Desktop/work/MSM107_analysis/data/ADCP_MSM107"
plotpath = "/Users/dkim/Desktop/work/MSM107_analysis/plots"

In [4]:
# grab data
adcp_38 = netCDF4.Dataset(os.path.join(adcppath, "MSM107_OS38_cor.nc"),"r+")
adcp_75 = netCDF4.Dataset(os.path.join(adcppath, "MSM107_OS75_cor.nc"),"r+")

In [ ]:
# looking at adcp data
print(adcp_75.variables.keys())
print(adcp_38)

In [5]:
# get data
depth_38 = adcp_38.variables["DEPH"][:]
time_38 = adcp_38.variables["DATE_TIME_UTC"][:]
sv_38 = adcp_38.variables['ECI'][:]
flag1_38 = adcp_38.variables['CAS_CURRENT_FLAG'][:]
UC_38 = adcp_38.variables['UVEL_ADCP'][:]
VC_38 = adcp_38.variables['VVEL_ADCP'][:]
lat_38 = adcp_38.variables['LATITUDE'][:]
lon_38 = adcp_38.variables['LONGITUDE'][:]

depth_75 = adcp_75.variables["DEPH"][:]
time_75 = adcp_75.variables["DATE_TIME_UTC"][:]
sv_75 = adcp_75.variables['ECI'][:]
flag1_75 = adcp_38.variables['CAS_CURRENT_FLAG'][:]
UC_75 = adcp_38.variables['UVEL_ADCP'][:]
VC_75 = adcp_38.variables['VVEL_ADCP'][:]
lat_75 = adcp_38.variables['LATITUDE'][:]
lon_75 = adcp_38.variables['LONGITUDE'][:]

In [6]:
# masking current data to remove low qualita data based on the flag
UC_mask_38 = np.ma.masked_where(flag1_38 > 1, UC_38)
VC_mask_38 = np.ma.masked_where(flag1_38 > 1, VC_38)

UC_mask_75 = np.ma.masked_where(flag1_75 > 1, UC_75)
VC_mask_75 = np.ma.masked_where(flag1_75 > 1, VC_75)

In [7]:
def nct2dt(ncttime):
    # convert nc datetime to datetime format
    t = ncttime.astype("str", copy = False) # convert elements in array to str
    x_len, y_len = t.shape[0], t.shape[1]
    empty_ary = np.zeros(shape=(x_len, 1))
    for x in range(0, x_len):
        empty_ary[x][0] = "".join(t[x]) # combining elements in an element
    l = list(itertools.chain(empty_ary)) # unpacking the list
    l_datetime = [datetime.strptime(str(int(d)), "%Y%m%d%H%M%S") for d in l] # converting element from str to datetime
    return l_datetime

def coord2NESW(x,y):
    # convert coordinate to polar direction
    # x = East and West
    # y = North and South
    theta = np.arctan2(x, y)
    theta = theta*180/math.pi
    DegN = 90-theta
    DegNW = DegN % 360 # direction
    return DegNW



detime_38 = nct2dt(time_38)
detime_75 = nct2dt(time_75)

In [8]:
# find datetime index by set datetime
def findidxdt(dtlist, startdt, enddt):
    idx_l = ([i for i, n in enumerate(dtlist) 
        if n >= datetime.strptime(startdt, "%Y%m%d%H%M%S") and n <= datetime.strptime(enddt, "%Y%m%d%H%M%S")])
    start_idx, end_idx = min(idx_l), max(idx_l)
    return start_idx, end_idx

idx_38_7 = findidxdt(detime_38, "202205240800", "202205241600")
idx_38_8 = findidxdt(detime_38, "202205250200", "202205250900")
idx_38_10 = findidxdt(detime_38, "202205252300", "202205260010")
idx_38_11 = findidxdt(detime_38, "202205261300", "202205261400")
idx_38_13 = findidxdt(detime_38, "202205270000", "202205270800")
idx_38_14 = findidxdt(detime_38, "202205271600", "202205272300")
idx_38_16 = findidxdt(detime_38, "202205281700", "202205281800")

idx_75_7 = findidxdt(detime_75, "202205240800", "202205241600")
idx_75_8 = findidxdt(detime_75, "202205250200", "202205250900")
idx_75_10 = findidxdt(detime_75, "202205252300", "202205260010")
idx_75_11 = findidxdt(detime_75, "202205261300", "202205261400")
idx_75_13 = findidxdt(detime_75, "202205270000", "202205270800")
idx_75_14 = findidxdt(detime_75, "202205271600", "202205272300")
idx_75_16 = findidxdt(detime_75, "202205281700", "202205281800")

tran1_38 = findidxdt(detime_38, "20220521211400", "20220525173400")
tran2_1_38 = findidxdt(detime_38, "20220525172200", "20220527031500")
tran2_2_38 = findidxdt(detime_38, "20220529152900", "20220529171700")
tran3_38 = findidxdt(detime_38, "20220527160800", "20220529062400")

tran1_75 = findidxdt(detime_75, "20220521211400", "20220525173400")
tran2_1_75 = findidxdt(detime_75, "20220525172200", "20220527031500")
tran2_2_75 = findidxdt(detime_75, "20220529152900", "20220529171700")
tran3_75 = findidxdt(detime_75, "20220527160800", "20220529062400")

In [12]:
# transection 1 plot
fig = plt.figure(figsize=(10, 12), facecolor="white")
spec = gridspec.GridSpec(ncols=1, nrows=4, figure=fig)

ax1 = fig.add_subplot(spec[0, 0])
im1 = ax1.contourf(lon_38[tran1_38[0]: tran1_38[1]], depth_38,VC_mask_38.T[:, tran1_38[0]: tran1_38[1]], cmap="RdBu", vmax=1., vmin=-1.)
fig.colorbar(im1, orientation='vertical', ax=ax1, label="m/s")
ax1.set_xlim(-10.7, -9.7)
ax1.set_title("V speed")
ax1.set_ylabel("Depth (m)")

ax2 = fig.add_subplot(spec[1, 0])
im2 = ax2.contourf(lon_38[tran1_38[0]: tran1_38[1]], depth_38, UC_mask_38.T[:, tran1_38[0]: tran1_38[1]], cmap="RdBu", vmax=1., vmin=-1.)
fig.colorbar(im2, orientation='vertical', ax=ax2, label="m/s")
ax2.set_xlim(-10.7, -9.7)
ax2.set_title("U speed")
ax2.set_ylabel("Depth (m)")


ax3 = fig.add_subplot(spec[2, 0])
im3 = ax3.contourf(lon_38[tran1_38[0]: tran1_38[1]], depth_38, np.sqrt( np.square(UC_mask_38.T[:, tran1_38[0]: tran1_38[1]]) + np.square(VC_mask_38.T[:, tran1_38[0]: tran1_38[1]]) ),
              cmap="gray_r",vmax=1.3, vmin=0.) 
fig.colorbar(im3, orientation='vertical', ax=ax3, label="m/s")
ax3.set_xlim(-10.7, -9.7)
ax3.set_title("Current speed")
ax3.set_xlabel("Longitude")
ax3.set_ylabel("Depth (m)")

direction = [coord2NESW(x,y) for x, y in zip(VC_mask_38.T[:, tran1_38[0]: tran1_38[1]],UC_mask_38.T[:, tran1_38[0]: tran1_38[1]])]
ax4 = fig.add_subplot(spec[3, 0])
im4 = ax4.contourf(lon_38[tran1_38[0]: tran1_38[1]], depth_38, direction,
              cmap="twilight", vmax=360, vmin=0.)
fig.colorbar(im4, orientation='vertical', ax=ax4, label="direction")
ax4.set_xlim(-10.7, -9.7)
ax4.set_title("Current direction")
ax4.set_xlabel("Longitude")
ax4.set_ylabel("Depth (m)")

fig.tight_layout()

plt.savefig(os.path.join(plt_path,'adcp_38_tran1.pdf'), dpi=600, facecolor="w")
plt.close()

In [13]:
# transection 3 plot
fig = plt.figure(figsize=(10, 12), facecolor="white")
spec = gridspec.GridSpec(ncols=1, nrows=4, figure=fig)

ax1 = fig.add_subplot(spec[0, 0])
im1 = ax1.contourf(lon_38[tran3_38[0]: tran3_38[1]], depth_38,VC_mask_38.T[:, tran3_38[0]: tran3_38[1]], cmap="RdBu", vmax=1., vmin=-1.)
fig.colorbar(im1, orientation='vertical', ax=ax1, label="m/s")
#ax1.set_xlim(-10.7, -9.7)
ax1.set_title("V speed")
ax1.set_ylabel("Depth (m)")

ax2 = fig.add_subplot(spec[1, 0])
im2 = ax2.contourf(lon_38[tran3_38[0]: tran3_38[1]], depth_38, UC_mask_38.T[:, tran3_38[0]: tran3_38[1]], cmap="RdBu", vmax=1., vmin=-1.)
fig.colorbar(im2, orientation='vertical', ax=ax2, label="m/s")
#ax2.set_xlim(-10.7, -9.7)
ax2.set_title("U speed")
ax2.set_ylabel("Depth (m)")


ax3 = fig.add_subplot(spec[2, 0])
im3 = ax3.contourf(lon_38[tran3_38[0]: tran3_38[1]], depth_38, np.sqrt( np.square(UC_mask_38.T[:, tran3_38[0]: tran3_38[1]]) + np.square(VC_mask_38.T[:, tran3_38[0]: tran3_38[1]]) ),
              cmap="gray_r",vmax=1.3, vmin=0.) 
fig.colorbar(im3, orientation='vertical', ax=ax3, label="m/s")
#ax3.set_xlim(-10.7, -9.7)
ax3.set_title("Current speed")
ax3.set_xlabel("Longitude")
ax3.set_ylabel("Depth (m)")

direction = [coord2NESW(x,y) for x, y in zip(VC_mask_38.T[:, tran3_38[0]: tran3_38[1]],UC_mask_38.T[:, tran3_38[0]: tran3_38[1]])]
ax4 = fig.add_subplot(spec[3, 0])
im4 = ax4.contourf(lon_38[tran3_38[0]: tran3_38[1]], depth_38, direction,
              cmap="twilight",vmax=360, vmin=0.) 
fig.colorbar(im4, orientation='vertical', ax=ax4, label="direction")
#ax4.set_xlim(-10.7, -9.7)
ax4.set_title("Current direction")
ax4.set_xlabel("Longitude")
ax4.set_ylabel("Depth (m)")


fig.tight_layout()

plt.savefig(os.path.join(plt_path,'adcp_38_tran3.pdf'), dpi=600, facecolor="w")
plt.close()

In [14]:
# transection 2 plot
fig = plt.figure(figsize=(10, 12), facecolor="white")
spec = gridspec.GridSpec(ncols=1, nrows=4, figure=fig)

ax1 = fig.add_subplot(spec[0, 0])
im1 = ax1.contourf(lon_38[ np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]  ], depth_38,
                VC_mask_38.T[:, np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]], cmap="RdBu", vmax=1., vmin=-1.)
fig.colorbar(im1, orientation='vertical', ax=ax1, label="m/s")
#ax1.set_xlim(-10.7, -9.7)
ax1.set_title("V speed")
ax1.set_ylabel("Depth (m)")

ax2 = fig.add_subplot(spec[1, 0])
im2 = ax2.contourf(lon_38[ np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1]]], depth_38, 
                UC_mask_38.T[:, np.r_[tran2_1_38[0]: tran2_1_38[1], tran2_2_38[0]: tran2_2_38[1]] ], cmap="RdBu", vmax=1., vmin=-1.)
fig.colorbar(im2, orientation='vertical', ax=ax2, label="m/s")
#ax2.set_xlim(-10.7, -9.7)
ax2.set_title("U speed")
ax2.set_ylabel("Depth (m)")


ax3 = fig.add_subplot(spec[2, 0])
im3 = ax3.contourf(lon_38[np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]], depth_38, 
            np.sqrt( np.square(UC_mask_38.T[:, np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]]) + 
            np.square(VC_mask_38.T[:, np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]]) ),
              cmap="gray_r",vmax=1.3, vmin=0.) 
fig.colorbar(im3, orientation='vertical', ax=ax3, label="m/s")
#ax3.set_xlim(-10.7, -9.7)
ax3.set_title("Current speed")
ax3.set_xlabel("Longitude")
ax3.set_ylabel("Depth (m)")

direction = [coord2NESW(x,y) for x, y in zip(VC_mask_38.T[:, np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]],
            UC_mask_38.T[:, np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]])]
ax4 = fig.add_subplot(spec[3, 0])
im4 = ax4.contourf(lon_38[np.r_[tran2_1_38[0]: tran2_1_38[1] , tran2_2_38[0]: tran2_2_38[1] ]], depth_38, direction,
              cmap="twilight",vmax=360, vmin=0.) 
fig.colorbar(im4, orientation='vertical', ax=ax4, label="direction")
#ax4.set_xlim(-10.7, -9.7)
ax4.set_title("Current direction")
ax4.set_xlabel("Longitude")
ax4.set_ylabel("Depth (m)")



fig.tight_layout()

plt.savefig(os.path.join(plt_path,'adcp_38_tran2.pdf'), dpi=600, facecolor="w")
plt.close()

In [11]:
# individual plot for adcp 38kHz
fig, ax = plt.subplots(2,2, figsize=(12, 12), facecolor="white", )

fig.suptitle("adcp 38kHz") # plot name

# profile 14
ax1 = ax[0,0]
ax1.contourf(detime_38[idx_38_14[0]: idx_38_14[1]], depth_38, sv_38.T[:, idx_38_14[0]: idx_38_14[1]])
ax1.set_title("profile14")
ax1.tick_params(axis='x', rotation=45)

# profile 13
ax2 = ax[0,1]
ax2.contourf(detime_38[idx_38_13[0]: idx_38_13[1]], depth_38, sv_38.T[:, idx_38_13[0]: idx_38_13[1]])
ax2.set_title("profile13")
ax2.tick_params(axis='x', rotation=45)

# profile 08
ax3 = ax[1,1]
ax3.contourf(detime_38[idx_38_8[0]: idx_38_8[1]], depth_38, sv_38.T[:, idx_38_8[0]: idx_38_8[1]])
ax3.set_title("profile08")
ax3.tick_params(axis='x', rotation=45)

# profile 07
ax4 = ax[1,0]
ax4.contourf(detime_38[idx_38_7[0]: idx_38_7[1]], depth_38, sv_38.T[:, idx_38_7[0]: idx_38_7[1]])
ax4.set_title("profile07")
ax4.tick_params(axis='x', rotation=45)

fig.tight_layout()

plt.savefig(os.path.join(plotpath,'adcp_38.pdf'), dpi=600, facecolor="w")
plt.close()

In [12]:
# individual plot for adcp 75kHz

fig, ax = plt.subplots(2,2, figsize=(12, 12), facecolor="white", )

fig.suptitle("adcp 38kHz") # plot name

# profile 14
ax1 = ax[0,0]
ax1.contourf(detime_75[idx_75_14[0]: idx_75_14[1]], depth_75, sv_75.T[:, idx_75_14[0]: idx_75_14[1]])
ax1.set_title("profile14")
ax1.tick_params(axis='x', rotation=45)

# profile 13
ax2 = ax[0,1]
ax2.contourf(detime_75[idx_75_13[0]: idx_75_13[1]], depth_75, sv_75.T[:, idx_75_13[0]: idx_75_13[1]])
ax2.set_title("profile13")
ax2.tick_params(axis='x', rotation=45)

# profile 08
ax3 = ax[1,1]
ax3.contourf(detime_75[idx_75_8[0]: idx_75_8[1]], depth_75, sv_75.T[:, idx_75_8[0]: idx_75_8[1]])
ax3.set_title("profile08")
ax3.tick_params(axis='x', rotation=45)

# profile 07
ax4 = ax[1,0]
ax4.contourf(detime_38[idx_75_7[0]: idx_75_7[1]], depth_75, sv_75.T[:, idx_75_7[0]: idx_75_7[1]])
ax4.set_title("profile07")
ax4.tick_params(axis='x', rotation=45)

fig.tight_layout()

plt.savefig(os.path.join(plotpath,'adcp_75.pdf'), dpi=600, facecolor="w")
plt.close()
